In [22]:
import polars as pl
import pickle

from hgm.config import MODELS_DIR, DATA_DIR

In [23]:
def map_positions(pos_column):
    return pos_column.replace({
        'C': 1,
        'W': 2,
        'D': 3,
        'G': 4
    }).cast(pl.Int32)

In [24]:
players_raw = pl.scan_parquet(DATA_DIR / 'raw' / 'players.parquet')
player_progs = pl.scan_parquet(DATA_DIR / 'constants' / 'calculateD_progs.parquet')

with open(DATA_DIR / 'raw' / 'game_settings.pkl', 'rb') as file:
    game_settings = pickle.load(file)

model_dict = {
    position: pickle.load(open(MODELS_DIR / 'ovr_to_cap' / f'{position}.pkl', 'rb'))
    for position in ['C', 'W', 'D', 'G']
}

salary_model = pickle.load(open(MODELS_DIR / 'salary' / 'xgboost_salary.pkl', 'rb'))

In [25]:
def load_players(players_raw, player_progs):
    player_ratings = (
        players_raw
        .with_columns(
            pl.concat_str(pl.col('firstName'), pl.lit(' '), pl.col('lastName')).alias('player'),
        )
        .explode('ratings')
        .unnest('ratings')
        .with_columns(
            age=pl.col('season') - pl.col('born').struct.field('year')
        )
        .unique(['pid', 'season'])
        .sort(['pid', 'season'])
        .select('player', 'pid', 'tid', 'season', 'age', 'pos', 'ovr')
        .filter(pl.col('season') >= game_settings['season'])
    )

    player_salaries = (
        players_raw
        .select('pid', 'salaries')
        .explode('salaries')
        .unnest('salaries')
        .group_by(['pid', 'season'])
        .agg(salary=pl.col('amount').last())
        .sort('pid', 'season')
    )

    ovr_progs = (
        player_progs
        .filter(pl.col('variable').str.contains('ovr'))
        .select(
            pl.col('pos'),
            pl.col('age'),
            pl.col('ovr'),
            pl.col('variable').str.split('_').list.last().cast(pl.Int64).add(game_settings['season']).alias('season'),
            pl.col('value').alias('ovr_pred')
        )
    )

    value_progs = (
        player_progs
        .filter(pl.col('variable').str.contains('value'))
        .select(
            pl.col('pos'),
            pl.col('age'),
            pl.col('ovr'),
            pl.col('variable').str.split('_').list.last().cast(pl.Int64).add(game_settings['season']).alias('season'),
            pl.col('value').alias('value_pred')
        )
    )

    players = (
        player_ratings
        .with_columns(
            pl.Series('season', [range(game_settings['season'], game_settings['season'] + 10)], dtype=pl.List)
        )
        .explode('season')
        .join(player_salaries, on=['pid', 'season'], how='left')
        .join(ovr_progs, on=['pos', 'age', 'season', 'ovr'], how='left')
        .join(value_progs, on=['pos', 'age', 'season', 'ovr'], how='left')
        .with_columns(
            cap_value=(
                pl.when(pl.col('pos') == 'C').then(
                    pl.col('ovr') * model_dict['C'].coef_[0] + model_dict['C'].intercept_[0])
                .when(pl.col('pos') == 'W').then(
                    pl.col('ovr') * model_dict['W'].coef_[0] + model_dict['W'].intercept_[0])
                .when(pl.col('pos') == 'D').then(
                    pl.col('ovr') * model_dict['D'].coef_[0] + model_dict['D'].intercept_[0])
                .when(pl.col('pos') == 'G').then(
                    pl.col('ovr') * model_dict['G'].coef_[0] + model_dict['G'].intercept_[0])
                .clip(0)
            ),
        )
        .select(
            'player', 'pid', 'tid', 'season', 'pos',
            age=pl.col('age') + pl.col('season') - game_settings['season'],
            salary=pl.col('salary') / 1000,
            ovr=pl.when(pl.col('season') == game_settings['season']).then(pl.col('ovr')).otherwise(pl.col('ovr_pred')),
            cap_value=pl.when(pl.col('season') == game_settings['season']).then(pl.col('cap_value')).otherwise(
                pl.col('value_pred'))
        )
    )

    return players

In [26]:
def add_placeholder_salaries(df):
    last_contracts = (
        df
        .filter(pl.col('salary').is_not_null())
        .group_by('pid').tail(1)
        .select('pid', 'season', map_positions(pl.col('pos')), 'age', 'ovr', 'salary')
        .with_columns(
            season=pl.col('season').map_elements(lambda x: list(range(x + 1, x + 6)), return_dtype=pl.List(pl.Int64))
        )
        .explode('season')
    )

    last_contracts = last_contracts.with_columns(
        on_last_contract=salary_model.predict(last_contracts.select(['pos', 'age', 'ovr', 'salary']))
    )

    no_contracts = (
        df
        .filter(pl.col('salary').first().over('pid').is_null())
        .group_by('pid').head(1)
        .select('pid', 'season', map_positions(pl.col('pos')), 'age', 'ovr', 'salary')
        .with_columns(
            season=pl.col('season').map_elements(lambda x: list(range(x, x + 5)), return_dtype=pl.List(pl.Int64))
        )
        .explode('season')
    )

    no_contracts = no_contracts.with_columns(
        on_no_contract=salary_model.predict(no_contracts.select(['pos', 'age', 'ovr', 'salary']))
    )

    df = (
        df
        .join(last_contracts.select('pid', 'season', 'on_last_contract'), on=['pid', 'season'], how='left')
        .join(no_contracts.select('pid', 'season', 'on_no_contract'), on=['pid', 'season'], how='left')
        .with_columns(
            salary_next=(
                pl.when(pl.col('salary').is_not_null()).then(None)
                .otherwise(pl.col('on_last_contract').fill_null(pl.col('on_no_contract')))
            )
        )
        .drop('on_last_contract', 'on_no_contract')
    )

    return df

In [27]:
def add_contract_value(cap_value, salary):
    return cap_value - salary

In [28]:
def load_and_process_players(players_raw, player_progs):
    players = (
        load_players(players_raw, player_progs)
        .collect()
        .pipe(add_placeholder_salaries)
        .lazy()
        .with_columns(
            pot=pl.col('ovr').max().over('pid'),
            cap_surplus=add_contract_value(pl.col('cap_value'), pl.col('salary')),
            next_cap_surplus=add_contract_value(pl.col('cap_value'), pl.col('salary_next'))
        )
        .with_columns(
            contract_value=pl.col('cap_surplus').sum().over('pid'),
            next_contract_value=pl.col('next_cap_surplus').sum().over('pid')
        )
        .with_columns(
            total_contract_value=pl.col('contract_value') + pl.col('next_contract_value').clip(0),
            status=(
                pl.when(pl.col('salary').is_not_null()).then(pl.lit('current'))
                .when(pl.col('salary_next').is_not_null()).then(pl.lit('next'))
                .otherwise(pl.lit('none'))
            ).cast(pl.Categorical)
        )
        .select(
            'player', 'pid', 'tid', 'season', 'pos', 'age', 'status', 'ovr', 'pot',
            value=pl.col('cap_value'),
            salary=(
                pl.when(pl.col('status') == 'current').then(pl.col('salary'))
                .when(pl.col('status') == 'next').then(pl.col('salary_next'))
                .otherwise(None)
            ),
            surplus=(
                pl.when(pl.col('status') == 'current').then(pl.col('cap_surplus'))
                .when(pl.col('status') == 'next').then(pl.col('next_cap_surplus'))
                .otherwise(None)
            ),
            cv_current=pl.col('contract_value'),
            cv_next=pl.col('next_contract_value'),
            cv_total=pl.col('total_contract_value'),
        )
        .collect()
    )
    return players

In [30]:
teams = pl.read_parquet(DATA_DIR / 'raw' / 'teams.parquet')

In [33]:
teams.select('tid','region','abbrev').to_pandas()

,tid,region,abbrev
0,0,Brooklyn,BKN
1,1,Boston,BOS
2,2,Buffalo,BUF
3,3,Calgary,CGY
4,4,Chicago,CHI
5,5,Columbus,CLB
6,6,Dallas,DAL
7,7,Denver,DEN
8,8,Detroit,DET
9,9,Edmonton,EDM
